# Baseline Model: Logistisk Regression

Udviklingen af en ML-model består typisk i at prøve en masse forskellige eksperimenter, predictors og model tuning af for at finde frem til den løsning, der performer bedst på ens data.

Det er *god praksis* i første iteration at udvikle en simpel *baseline*-model, der kan fungere som benchmark for - mere avancerede - model-arkitekturer i senere iterationer.

I denne notebook udvikler vi en *baseline*-model, der anvender velkendt *logistisk regression* som predictor.

## Import af python-afhængigheder

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from feature_engine.selection import (
    DropConstantFeatures,
    DropCorrelatedFeatures,
)
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from workshop.utils import DropFeatures

Som det ses, kommer vi til at anvende mange værktøjer fra pakken [scikit-learn](https://scikit-learn.org/stable/index.html) (=`sklearn`), der (fortsat) er det førende framework til generel Machine Learning i Python.

Specielt kommer vi til at bruge [`sklearn.pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), der giver en stringent programmatisk struktur til at definere den samlede "opskrift" på vores ML-model.

Til sidst anvender vi "opskriften" til at træne vores samlede model.

**ADVARSEL**: at udvikle en model med `sklearn.pipeline` kan umiddelbart virke ret tungt og besværligt, og det kan måske være svært at se, hvad man får ud af det. Vi vender tilbage til, hvorfor det er attraktivt.

## Indlæs data

In [ ]:
# indlæs "rådata" for 2015-18
df = pd.read_parquet("data/train.parquet")

# beregn binomialt target, der angiver, om der har været skade(r) eller ej
y = df["skadesantal"].values > 0

# instantiér generator til split af data, 'random_state' sættes for at kunne reproducere split
split_generator = ShuffleSplit(n_splits=1, train_size=0.8, random_state=42)

# generér indices for de to datasæt
train_index, val_index = next(split_generator.split(df))

# opdel data i trænings- og valideringssæt
X_train, y_train = df.iloc[train_index], y[train_index]
X_val, y_val = df.iloc[val_index], y[val_index]

## Data-transformationer

En (meget) stor del af arbejdet med at udvikle en ML-model består typisk i at definere de transformationer, som data skal gennemgå, førend de fødes ind i selve predictoren. Denne proces kaldes i flæng også for "data preprocessing" og "feature engineering".

Vi anvender til dette formål en række [`transformers`](https://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html#sklearn.base.TransformerMixin), der hver især transformerer data - typisk på baggrund af automatiseret "læring" fra træningssættet.

### Drop variable
Først skridt i vores data-transformations-pipeline er at fjerne uønskede features.

For eksempel bør vi indlysende fjerne features, der indeholder direkte information om vores target, da der ellers ville være tale om snyd, og vi ville ende ud med en ubrugelig model.

Vi instantierer til det formål en simpel `DropFeatures`-transformer, der blot fjerner features fra en liste med brugerdefinerede, hardcodede navne på features: `blacklist` gennem sin `transform()`-metode.

In [ ]:
blacklist = [
    # 'skadesudgift' og 'skadesantal' er direkte afledt af target-variablen, og kan derfor ikke bruges til at prædiktere den
    "skadesudgift",
    "skadesantal",
    # vi fjerner 'aar', da vi indtil videre bortser fra eventuelle tidslige mønstre i data
    "aar",
    # 'idnummer' er en unik identifikator for hver kunde, der ikke - i sig selv - indeholder signal
    "idnummer",
]
drop_features = DropFeatures(blacklist)

# demonstrér transformer
print(
    "Tilbageværende features efter transformation:",
    drop_features.transform(X_train).columns,
)

God ML-skik tilsiger, at omfanget af "hard-coded" variabel-fravalg - som ovenstående - skal holdes på et minimum. En tommelfingerregel er, at jo færre "hardcodede"-variabelnavne der er i koden, desto bedre.

I stedet ønsker man, at en eventuel deselektions-proces for variable i videst muligt omfang skal *læres* fra (/fittes på) vores *træningssæt*, og at den skal kunne generalisere til eventuelle nye variable.

Et af de mest simple eksempler på en *tillært* variabel-deselektion, er at fjerne de variable, der er konstante i *træningssæt*, og derfor ikke vil bidrage med noget godt til vores predictor.

Vi bruger en [`DropConstantFeatures`](https://feature-engine.trainindata.com/en/latest/api_doc/selection/DropConstantFeatures.html)-transformer, der gennem sin `fit()`-metode "lærer"/identificerer disse variable på træningssættet. Herefter kan transformeren - med `transform()` - transformerer nye datasæt og observationer herunder vores *validerings-* og *testsæt*.

Nedenfor ses et eksempel på virkemåden for `DropConstantFeatures`.

In [ ]:
# instantiér transformer
drop_constant_features = DropConstantFeatures(missing_values="ignore")

# afprøv transformer på træningsdata
# lær/identificér først features, der er konstante i træningsdata
drop_constant_features.fit(X_train)

# transformeren identificerer følgende variable som konstante
removed_vars = list(
    set(drop_constant_features.feature_names_in_)
    - set(drop_constant_features.get_feature_names_out())
)
print(
    "Transformeren identificerer følgende features til at være konstante i træningsdata:",
    removed_vars,
)

# Herefter kan transformeren anvendes på et vilkårligt datasæt, hvorfra den fjerner ovenstående variable
drop_constant_features.set_output(transform="pandas")
print(
    "Tilbageværende features efter transformation:",
    drop_constant_features.transform(X_train).columns,
)

Herudover kan det give problemer for vores logistiske regression, hvis nogle af vores features er meget højt indbyrdes korrelerede. 

Derfor vil vi i vores pipeline også anvende en [`DropCorrelatedFeatures`](https://feature-engine.trainindata.com/en/latest/api_doc/selection/DropCorrelatedFeatures.html)-transformer, der identificerer og fjerner variable, der er meget højt indbyrdes korrelerede.


### Feature-transformationer

Valget af predictor kan også have betydning for, om/hvordan de enkelte features i datasættet skal transformeres. Nogle predictors kan håndtere stort set alt, mens andre predictors - såsom logistisk regression - ikke accepterer missing-værdier. Det sidste gælder også for vores logistiske regression. 

Missing-værdier kan håndteres ved imputation, hvilket vi vil gøre i vores pipeline.

Nedenfor definerer vi, hvordan hhv. numeriske og kategoriske features skal transformeres.

#### Numeriske features

Vi ønsker at imputere *missing*-værdier for numeriske features til variablernes respektive median-værdier.

Derudover kan det erfaringsmæssigt for en logistisk regression være [nyttigt at standardisere/skalere de numeriske features]((https://forecastegy.com/posts/does-logistic-regression-require-feature-scaling/)), så de har middelværdi $\mu=0$ og standardafvigelse $\sigma=1$:

$$z = \frac{x - \mu}{\sigma}$$

Vi sætter de to operationer sammen til en pipeline, som vi ønsker at anvende på *alle numeriske features*.

In [ ]:
transformer_num = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

# afprøv pipeline på et udsnit af numeriske features i datasættet
print(
    "Træningsdata før transformation:\n",
    X_train.select_dtypes(include="number").describe(),
)

# fit transformer på træningsdata
transformer_num.set_output(transform="pandas")
transformer_num.fit(X_train.select_dtypes(include="number"))

# transformér træningsdata
print(
    "Træningsdata efter transformation:\n",
    transformer_num.transform(X_train.select_dtypes(include="number")).describe(),
)

#### Kategoriske features

Missing-værdier for kategoriske features ønsker vi at imputere til de respektive variablers hyppigst forekommende værdi i træningssættet.

Da logistisk regression kun accepterer numeriske variable, specificerer vi desuden, at de kategoriske variable skal ["one-hot"-encodes](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

In [ ]:
transformer_cat = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "encoder",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first"),
        ),
    ]
)

# afprøv pipeline på en enkelt variabel, "geo"
geo = X_train[["geo"]]
print(
    "variablen 'geo' før transformation:\n",
    geo,
    "\n#missings:",
    geo.isna().sum().sum(),
)

transformer_cat.set_output(transform="pandas")
geo_transformed = transformer_cat.fit_transform(X_train[["geo"]])
print(
    "variablen 'geo' efter transformation:\n",
    geo_transformed,
    "\n#missings:",
    geo_transformed.isna().sum().sum(),
)

Pr. `sklearn.pipeline` konvention skal feature-specifikke transformationer defineres i en såkaldt [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html). 

Med en `ColumnTransformer` har brugeren mulighed for at specificere, hvilke features, der konkret skal transformeres.

Det er således i vores `ColumnTransformer`, vi specificerer, at de to ovenstående pipelines skal anvendes på hhv. alle numeriske og alle kategoriske features.

In [ ]:
col_transformer = ColumnTransformer(
    transformers=[
        (
            "transformer_num",
            transformer_num,
            make_column_selector(dtype_include=["number"]),
        ),
        (
            "transformer_cat",
            transformer_cat,
            make_column_selector(dtype_include=["category"]),
        ),
    ],
    verbose_feature_names_out=False,
)
# bemærk, vi bruger 'make_column_selector' dynamisk - altså når transformeren fittes - til at identificere features af en given type

# sæt output-typen for transformeren til pandas.DataFrame for bedre at kunne inspicere output (ellers returnerer den et array, der er sværere at afkode)
col_transformer.set_output(transform="pandas")

# vis resulterende column transformer
col_transformer

### Komplet data-transformations-pipeline
De ovenstående transformationer vil tilsammen kunne transformere vores træningsdata, så en logistisk regression kan fittes på de transformerede data.

Vi sætter alle de ovenstående transformationer sammen til en samlet data-transformations-pipeline. 

In [ ]:
pipe = Pipeline(
    [
        ("drop_features_blacklist", DropFeatures(blacklist)),
        ("drop_constant_features", DropConstantFeatures(missing_values="ignore")),
        ("col_transformer", col_transformer),
        (
            "drop_correlated_features",
            DropCorrelatedFeatures(threshold=0.95, missing_values="ignore"),
        ),
    ]
)

# vis pipeline
pipe

Herefter kan vi let fitte den samlede pipeline på træningsdata, og derefter anvende den - også på nye datasæt og observationer.

Når pipelinen fittes, læres de parametre, der skal bruges til de underliggende transformers f.eks. $\mu$, $\sigma$, medianer, mest frekvente kategorier osv. på træningsdatasættet.

In [ ]:
# fit pipeline på de første 1000 observationer i træningssættet
pipe.fit(X_train.head(1000))

# undersøg, hvilke features der fjernes af "drop_correlated_features"-transformeren
corr_features = list(
    set(pipe.named_steps["drop_correlated_features"].feature_names_in_)
    - set(pipe.named_steps["drop_correlated_features"].get_feature_names_out())
)
print("Følgende features fjernes af 'drop_correlated_features'-transformeren:", corr_features)

# anvend pipeline på valideringssættet til illustration
print("Valideringssættet efter transformation:")
pipe.transform(X_val.head())

## Model-træning
Med data-transformationerne på plads kan vi nu vende blikket mod vores predictor.

Vi anvender `sklearn`'s implementation af [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Vi udvider derfor vores pipeline med denne predictor.

In [ ]:
# instantiér LogisticRegression-predictor
# vi skruer op for 'max_iter', da vi har relativt mange features
# vi bruger en "newton-cholesky"-solver, der ifølge dokumentationen skulle være god til at håndtere datasæt med vores karakteristika
predictor = LogisticRegression(max_iter=1000, solver="newton-cholesky")

# udvid pipeline med vores predictor
pipe.steps.append(["predictor", predictor])

# visualize pipeline
pipe

Nu kan vores pipeline inkl. predictor fittes under ét på vores træningsdata.

In [ ]:
pipe.fit(X_train, y_train);

Dermed har vi succesfuldt "trænet" vores første model! &#127881;

Denne model definerer vi som vores *baseline-model*.

## Model-evaluering

Efter modellen er fittet, evalueres dens performance.

Vi evaluerer både modellens prædiktive performance på træningssættet, som modellen er trænet på, og valideringssættet, som *modellen ikke har haft adgang til under træningen*. Performance på valideringssættet giver os en fornemmelse af, hvor godt modellen generaliserer til nye usete, data.

Vi plotter &#128073; [ROC-kurverne](slides/05_roc_auc.ipynb) for modellen på trænings- og valideringssættet.

In [ ]:
# beregn true positive rate og false positive rate for trænings- og valideringssæt
# bemærk, når vi bruger pipe.predict, anvendes hele pipelinen (dvs. fittede transformers + fitted predictor) på input
ns_fpr, ns_tpr, _ = roc_curve(y_train, [0 for _ in range(len(y_train))])
train_fpr, train_tpr, _ = roc_curve(y_train, pipe.predict_proba(X_train)[:, 1])
test_fpr, test_tpr, _ = roc_curve(y_val, pipe.predict_proba(X_val)[:, 1])

# plot ROC-kurver
plt.plot(ns_fpr, ns_tpr, linestyle="--", label="Naïve")
plt.plot(train_fpr, train_tpr, label="Train")
plt.plot(test_fpr, test_tpr, label="Validation")

# formatér og display plot
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

Vi bemærker, at ROC-kurverne for træning- og valideringssættet forløber næsten identisk.

Vi beregner for begge sæt [*ROC AUC*](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score), som er den metrik, vi bestræber os på at optimere modellen i forhold til.

In [ ]:
# beregn ROC AUC på både trænings- og valideringssæt
auc_train_baseline = roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1])
auc_val_baseline = roc_auc_score(y_val, pipe.predict_proba(X_val)[:, 1])
print("ROC AUC train: ", auc_train_baseline, "\nROC AUC val: ", auc_val_baseline)

Det bemærkes, at 

1. Modellen performer signifikant bedre end en naiv model (=tilfældigt gætteri) på både trænings- og valideringssæt
2. Der er ikke nogen tegn på, at modellen *overfitter*

### Model-diagnostik

Vi vil ikke bruge tid på det her på workshoppen, men hvis man ønsker at komme et spadestik dybere ned i den fittede predictor, vil Data Scientists typisk inspicere model-egenskaber såsom "Feature Importance".

For `LogisticRegression` er det nærmeste, man kommer det, en udskrift af de estimerede koefficienter.

Da alle numeriske variable er standardiserede, kan den absolutte værdi af deres koefficienter med tolkes som udtryk for deres "importance".

In [ ]:
pd.DataFrame(
    {
        "parameter": pipe.named_steps[
            "drop_correlated_features"
        ].get_feature_names_out(),
        "coefficient": pipe._final_estimator.coef_[0],
    }
)

I Machine Learning er man ikke som i traditionel statistik interesseret i inferens på feature-niveau, men langt mere i modellens prædiktive performance.

Det er også oplagt eksempelvis at inspicere fordelingen af de prædikterede sandsynligheder.

In [ ]:
sns.histplot(pipe.predict_proba(X_train)[:, 1])
plt.xlim(0, 0.1)

Det ser tilforladeligt ud (?)

Af andre nyttige plots og metrikker kan blandt andet nævnes [kalibreringsplots](https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html), [precision/recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall), [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) og [confusion matrix](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#confusion-matrix).

## Model tuning
Da vi nu er i stand til at træne vores model, udestår sidste skridt - nemlig at &#128073; [*tune*](slides/07_tuning.ipynb) den.

scikit-learns `LogisticRegression`-predictor implementerer som default [en tabsfunktion med regularisering](https://scikit-learn.org/stable/modules/linear_model.html#mathematical-details-3), der har regulariserings-parameteren $C$. Jo lavere/højere $C$, desto mere/mindre regularisering.

Vi prøver derfor at tune modellens parameter $C$ med henblik på at forbedre modellens performance.

Vi anvender et simpelt `GridSearch` til formålet.

In [ ]:
# Definér parameter-grid, der skal gennemsøges
param_grid = {
    "predictor__C": [1e-5, 1, 1e5],
}

# Forbered GridSearch
# Bemærk, at vi bruger vores split_generator til at generere trænings- og valideringssæt
grid = GridSearchCV(
    pipe,
    param_grid,
    cv=split_generator,
    scoring="roc_auc",
    return_train_score=True,
    n_jobs=-1,
    verbose=4,
)

# Udfør grid search - bemærk, vi giver hele datasættet (2015-18) som input og overlader det til split-generatoren at splitte det i trænings- og valideringssæt
grid.fit(df, y)

print("Disse parametre giver den bedste performance: ", grid.best_params_)
print("Med en ROC AUC på: ", grid.best_score_)
auc_val_iter_2 = grid.best_score_

# Udskriv resultater
results = pd.DataFrame.from_records(grid.cv_results_["params"])
results["mean_valid_score"] = grid.cv_results_["mean_test_score"]
results["mean_train_score"] = grid.cv_results_["mean_train_score"]
print("Resultater for grid search:")
results.sort_values("mean_valid_score", ascending=False)

Det ses, at der ikke er noget at hente ved øget regularisering. Det giver mening, fordi der ikke er nogen tegn på, at modellen skulle være  &#128073; [overfittet](slides/06_overfitting.ipynb).

En stor styrke ved, at vi har skrevet vores samlede model som en `sklearn.pipeline` er, at det gør det let at tune predictorens og data-transformationernes hyperparametre under ét.

I eksemplet nedenfor tuner vi til illustration $C$ og strategien for vores imputation af numeriske variable under ét.

In [ ]:
# Definér parameter-grid, der skal gennemsøges
param_grid = {
    "predictor__C": [1e-5, 1],
    "col_transformer__transformer_num__imputer__strategy": ["mean", "median"],
}

# Forbered GridSearch
grid = GridSearchCV(
    pipe,
    param_grid,
    cv=split_generator,
    scoring="roc_auc",
    return_train_score=True,
    n_jobs=-1,
    verbose=4,
)

# Udfør grid search
grid.fit(df, y)

print("Disse parametre giver den bedste performance: ", grid.best_params_)
print("Med en ROC AUC på: ", grid.best_score_)
auc_val_iter_3 = grid.best_score_

# Udskriv resultater
results = pd.DataFrame.from_records(grid.cv_results_["params"])
results["mean_valid_score"] = grid.cv_results_["mean_test_score"]
results["mean_train_score"] = grid.cv_results_["mean_train_score"]
print("Resultater for grid search:")
results.sort_values("mean_valid_score", ascending=False)

Vi konstaterer, at vi med ovenstående eksperimenter ikke har været i stand til at forbedre vores model ift. *baseline*-modellen.

Vi vedtager, at vi ikke vil investere mere energi i at tune modellen.

Vores bedste resultat var en ROC AUC på følgende:

In [ ]:
print("Bedste ROC AUC: ", max(auc_val_baseline, auc_val_iter_2, auc_val_iter_3))

## **NB.** Bemærkninger om `sklearn.pipeline`

Som sagt er der mange gode grunde til, hvorfor man bør skrive sin modelkode, som vi har gjort - altså med en `sklearn.pipeline`.

Efter at have afprøvet teknikken i eksemplet ovenfor, kan man forhåbentligvis give mening til argumenterne, der er sammenfattet i &#128073; [disse slides](slides/10_pipelines.ipynb).